# __Search Text by Text__

- Tutorial Difficulty: ★★☆☆☆
- 7 min read 
- Languages: [SQL](https://en.wikipedia.org/wiki/SQL) (100%)
- File location: tutorial_en/thanosql_search/search_text_by_text.ipynb   
- References: [(Kaggle) IMDB Movie Reviews](https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews/data), [Word Embeddings: LEXICAL SEMANTICS Encoding](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

## Tutorial Introduction

<div class="admonition note">
    <h4 class="admonition-title">Understanding Text Vectorization</h4>
    <p>Computers cannot directly interpret human language(natural language). Therefore, a process for converting natural language into numerical data that can be recognized by computers is required. In the field of natural language processing, embedding refers to the result of converting human natural language into a vectorized format, which is a form that can be understood by machines.</p>
</div>

Techniques for converting natural language into embeddings are largely divided into statistical techniques and artificial neural network-based techniques. ThanoSQL provides a method to train a text vectorization model using self-supervised learning.

<div class="admonition note">
    <h4 class="admonition-title">In This Tutorial</h4>
    <p>👉 Uses movie review data. The data consists of movie review texts and label values. However, because we are demonstrating self-supervised learning, this tutorial does not use label values. By training a model with 4,000 movie reviews, we will be able to search text by text and extract the importance of each word from a given movie review. </p>
</div>

## __0. Prepare Dataset__

As mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/how_to_use_ThanoSQL/#5-thanosql-workspace), you must create an API token and run the query below to execute the query of ThanoSQL. 

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

### __Prepare Dataset__

In [2]:
%%thanosql
GET THANOSQL DATASET movie_review_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" downloads the specified dataset to the workspace.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>GET THANOSQL DATASET</strong> clause.
        <ul>
            <li>"overwrite": determines whether to overwrite a dataset if it already exists. If set as True, the old dataset is replaced with the new dataset (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY movie_review_train
OPTIONS (overwrite=True) 
FROM 'thanosql-dataset/movie_review_data/movie_review_train.csv'

Success


In [4]:
%%thanosql
COPY movie_review_test 
OPTIONS (overwrite=True) 
FROM 'thanosql-dataset/movie_review_data/movie_review_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>COPY</strong>" specifies the name of the dataset to be saved as a database table.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>COPY</strong> clause.
        <ul>
           <li>"overwrite": determines whether to overwrite a table if it already exists. If set as True, the old table is replaced with the new table (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

## __1. Check Dataset__

To create a movie review text search model, we use the __movie_review_train__ table located in the ThanoSQL workspace database. Run the query below to check the contents of the table.

In [5]:
%%thanosql
SELECT *
FROM movie_review_train
LIMIT 5 

,review,sentiment
0,This is the kind of movie that BEGS to be show...,negative
1,Bulletproof is quite clearly a disposable film...,negative
2,A beautiful shopgirl in London is swept off he...,positive
3,"VERY dull, obvious, tedious Exorcist rip-off f...",negative
4,Do we really need any more narcissistic garbag...,negative


<div class="admonition note">
   <h4 class="admonition-title">Understanding the Data Table</h4>
   <p>The <strong>movie_review_train</strong> table contains the following information.</p>
   <ul>
      <li>review: movie review in text format</li>
      <li>sentiment: target value indicating whether the review has a positive or negative sentiment</li>
   </ul>
</div>

## __2. Build a Text Vectorization Model__

To create a text search model with the name __movie_text_search_model__ using the __movie_review_train__ table, run the following query.  
(Estimated duration of query execution: 2 min)

In [6]:
%%thanosql
BUILD MODEL movie_text_search_model
USING SBERTEn
OPTIONS (
    text_col='review',
    overwrite=True
    )
AS
SELECT *
FROM movie_review_train

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" creates and trains a model named <strong>movie_text_search_model</strong>.</li>
        <li>"<strong>USING</strong>" specifies <strong>SBERTEn</strong> as the base model.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values used to create a model.
        <ul>
            <li>"text_col": a column containing movie review data in the data table (str, default: 'text')</li>
            <li>"max_epochs": number of times to train with the training dataset (int, optional, default: 1)</li>
            <li>"batch_size": the size of dataset bundle utilized in a single cycle of training (int, optional, default: 16)</li> 
            <li>"overwrite": determines whether to overwrite a model if it already exists. If set as True, the old model is replaced with the new model (bool, optional, True|False, default: False) </li>
        </ul>
        </li>
    </ul>
</div>

To vectorize the __movie_review_test__ texts run the following "__CONVERT USING__" query. The vectorized results are stored in a user-defined column(default: 'convert_result') in the __movie_review_test__ table.

In [7]:
%%thanosql
CONVERT USING movie_text_search_model
OPTIONS (
    text_col='review',
    table_name='movie_review_test',
    batch_size=32,
    result_col="convert_result"
    )
AS 
SELECT *
FROM movie_review_test

,review,sentiment,convert_result
0,"I read the book before seeing the movie, and t...",positive,"[-0.0023354087, -0.0098618865, 0.039029818, -0..."
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,"[0.03316035, -0.04131365, 0.051284816, -0.0332..."
2,"Yesterday I attended the world premiere of ""De...",positive,"[0.008419235, -0.061199963, -0.0012992166, 0.0..."
3,Moonwalker is a Fantasy Music film staring Mic...,positive,"[0.016675944, -0.0081682615, 0.0453982, 0.0037..."
4,"Welcome to Oakland, where the dead come out to...",positive,"[-0.010290039, -0.101762444, -0.00702698, -0.0..."
...,...,...,...
995,Ocean's 12 starts off on annoying and gets wor...,negative,"[0.0076622674, -0.019105354, 0.019216133, -0.0..."
996,I remember catching this movie on one of the S...,negative,"[-0.011346059, -0.05119067, 0.009508793, 0.029..."
997,CyberTracker is set in Los Angeles sometime in...,negative,"[-0.0036547638, -0.04229665, 0.016640306, 0.00..."
998,"There is so much that is wrong with this film,...",negative,"[-0.02002796, 0.016561218, 0.019428799, -0.032..."


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" uses <strong>movie_text_search_model</strong> as an algorithm for text vectorizaion.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for text vectorizaion.
        <ul>
            <li>"text_col": a column containing movie review data in the data table (str, default: 'text')</li>
            <li>"table_name": the table name to be stored in the ThanoSQL workspace database. If a previously used table is specified, the existing table will be replaced by the new table with a 'convert_result' column. If not specified, the result dataframe will not be saved as a data table (str, optional)</li>
            <li>"batch_size": the size of dataset bundle utilized in a single cycle of training (int, optional, default: 16)</li>
            <li>"result_col": defines the column name that contains the vectorized results (str, optional, default: 'convert_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __3. Search for Similar Texts__

This step uses the __movie_text_search_model__ text vectorization model and test table to search for similar texts.

In [8]:
%%thanosql
SELECT review, sentiment, score
FROM (
    SEARCH TEXT 
    USING movie_text_search_model
    OPTIONS (
        search_by='text',
        search_input='This movie was my favorite movie of all time',
        emb_col='convert_result',
        result_col='score',
        top_k=10
        )
    AS 
    SELECT * 
    FROM movie_review_test
    )

,review,sentiment,score
0,I have loved this movie since I saw it in the ...,positive,0.666778
1,This movie was the second movie I saw on the c...,positive,0.640435
2,I saw this movie for the first time in 1988 wh...,positive,0.636465
3,This is a good film. This is very funny. Yet a...,positive,0.616408
4,This movie is Jackie's best. I still cant get ...,positive,0.604153
5,This movie was very good because it remember w...,positive,0.603827
6,This is a delightful film. Elizabeth Taylor do...,positive,0.602104
7,I saw this movie when I was about 12 when it c...,negative,0.601703
8,The Muppet movie is an instant classic. I reme...,positive,0.600884
9,"I can't explain it, but I find this movie not ...",positive,0.598664


In [9]:
%%thanosql
SELECT review, sentiment, score
FROM (
    SEARCH TEXT 
    USING movie_text_search_model
    OPTIONS (
        search_by='text',
        search_input='The movie was unsatisfactory',
        emb_col='convert_result',
        result_col='score',
        top_k=10
        )
    AS 
    SELECT * 
    FROM movie_review_test
    )

,review,sentiment,score
0,Just plain terrible. Nick and Michael are WAY ...,negative,0.691648
1,"Well, I remember when the studio sacked Schrad...",negative,0.675549
2,"If this movie would have been in English, all ...",negative,0.669978
3,Badly made. Dreadful acting and an ending that...,negative,0.665740
4,"We brought this film as a joke for a friend, a...",negative,0.661231
5,After having red the overwhelming reviews this...,negative,0.657562
6,To quote Clark Griswold (in the original Chris...,negative,0.650739
7,"I knew this movie wasn't going to be amazing, ...",negative,0.650328
8,"A gave it a ""2"" instead of a ""1"" (awful) becau...",negative,0.645362
9,There was absolutely nothing in this film that...,negative,0.644577


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|audio|video|text|keyword]</strong>" defines the image|audio|video|text data type to search for.</li>
        <li>"<strong>USING</strong>" defines the model used for the text vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for text searching.
        <ul>
            <li>"search_by": defines the image|text|audio|video type to be used for the search (str)</li>
            <li>"search_input": defines the input to be used for the search (str)</li>
            <li>"emb_col": the column that contains the vectorized results (str)</li>
            <li>"result_col": defines the name of the column that contains the search results (str, optional. default: 'search_result')</li>
            <li>"top_k": number of rows to return. If set as None, returns the entire data table (int, optional, default: 1000)</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for the searches. In this example, <strong>movie_review_test</strong> table is used.</li>
    </ul>
</div>

## __4. Extract Keywords from Texts__

This step uses the __movie_text_search_model__ text vectorization model and test table to extract keywords from the texts.

In [10]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col='review',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT * 
FROM movie_review_test
LIMIT 10 OFFSET 40

,review,sentiment,convert_result,keyword
0,"Like an earlier commentor, I saw it in 1980 an...",positive,"[-0.00405461, -0.06929114, -0.0052902964, -0.0...","{'keyword': ['story splendid acting', 'writers..."
1,"I'm a fan of Matthew Modine, but this film--wh...",negative,"[0.00012730403, -0.0063330038, 0.013847791, -0...","{'keyword': ['watched came film', 'absolutely ..."
2,Paul Lukas played a Russian intellectual makin...,positive,"[0.013866418, -0.071321495, -0.016819168, -0.0...","{'keyword': ['1930s bridge players', 'wonderfu..."
3,"to be honest, i didn't watch all of the origin...",negative,"[0.022904992, -0.017420417, 0.015385416, 0.004...","{'keyword': ['vampire movie', 'seven horrible ..."
4,Police Squad! (1982) was a funny show that end...,positive,"[-0.011529794, -0.04359309, 0.036978174, -0.00...","{'keyword': ['american television shows', 'new..."
5,I am still shuddering at the thought of EVER s...,negative,"[-0.016951086, -0.008924728, 0.025425913, -0.0...","{'keyword': ['performing worse movies', 'plot ..."
6,Gregory Peck gives a brilliant performance in ...,positive,"[0.0172633, 0.017774148, -0.007913495, -0.0598...","{'keyword': ['brilliant performance film', 'fi..."
7,I first flicked onto the LoG accidentally one ...,positive,"[-0.005874709, -0.014073846, 0.028541422, -0.0...","{'keyword': ['episode humour', 'watch like ll'..."
8,I didn't know what to make of this film. I gue...,positive,"[-0.023556368, -0.006610179, 0.029546514, -0.0...","{'keyword': ['view film think', 'appreciate re..."
9,A family looking for some old roadside attract...,negative,"[0.017549682, -0.05335354, -0.0059139486, -0.0...","{'keyword': ['forgettable poorly horror', 'bel..."


In [11]:
%%thanosql
SELECT review, sentiment, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD 
    USING movie_text_search_model
    OPTIONS (
        text_col='review',
        use_stopwords=True
        )
    AS 
    SELECT * 
    FROM movie_review_test
    LIMIT 10
    )

,review,sentiment,keywords,score
0,"I read the book before seeing the movie, and t...",positive,"[film haunting, best adaptations, faithful boo...","[0.4543, 0.3897, 0.3625, 0.3184, 0.2972]"
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,"[television, 11 2001, robert deniro, hosted, r...","[0.4391, 0.3715, 0.3368, 0.2824, 0.2237]"
2,"Yesterday I attended the world premiere of ""De...",positive,"[rape scene, innocent flirtation, sweethearts ...","[0.4082, 0.3504, 0.2927, 0.2659, 0.2536]"
3,Moonwalker is a Fantasy Music film staring Mic...,positive,"[loved montage, rating movie, thriller feel, c...","[0.4687, 0.4354, 0.3896, 0.3256, 0.2613]"
4,"Welcome to Oakland, where the dead come out to...",positive,"[carpenter haunting, bourgeois melodramas, ric...","[0.4625, 0.4011, 0.3771, 0.3744, 0.2728]"
5,Tipping the Velvet (2002) (TV) was directed by...,positive,"[wonderful acting, protagonist nan, rachael st...","[0.5159, 0.3942, 0.3708, 0.3568, 0.3106]"
6,The Stock Market Crash of 1929 and the Depress...,positive,"[musicals career, cagney favorite, staging bus...","[0.4379, 0.3811, 0.3673, 0.3264, 0.3199]"
7,I want to clarify a few things. I am not famil...,negative,"[art cinema, shocking scenes, commercial succe...","[0.5251, 0.4532, 0.4055, 0.3033, 0.281]"
8,This is a nice movie with good performances by...,positive,"[spanish cinema, good performances, movie bori...","[0.5879, 0.4773, 0.3978, 0.2694, 0.2292]"
9,"Once a month, I invite a few friends over for ...",negative,"[movie night, film indecisive, zombie personal...","[0.5551, 0.467, 0.3796, 0.364, 0.2589]"


In [12]:
%%thanosql
SELECT review, sentiment, json_array_elements(keyword -> 'keyword') AS keywords, json_array_elements(keyword -> 'score') AS score
FROM (
    SEARCH KEYWORD 
    USING movie_text_search_model
    OPTIONS (
        text_col='review',
        use_stopwords=True,
        threshold=0.5
        )
    AS 
    SELECT * 
    FROM movie_review_test
    LIMIT 10
    )

,review,sentiment,keywords,score
0,Tipping the Velvet (2002) (TV) was directed by...,positive,wonderful acting,0.5159
1,I want to clarify a few things. I am not famil...,negative,art cinema,0.5251
2,This is a nice movie with good performances by...,positive,spanish cinema,0.5879
3,"Once a month, I invite a few friends over for ...",negative,movie night,0.5551


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD</strong>" uses the <strong>movie_text_search_model</strong> as an algorithm to search keywords.</li>
        <li>"<strong>USING</strong>" defines the model used for the text vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for the text vectorizaion.
            <ul>
                <li>"lang": language to use (str, optional, 'ko'|'en' default: 'ko')</li>
                <li>"text_col": a column containing movie review data in the data table (str, default: 'text')</li>
                <li>"ngram_range": minimum and maximum number of words for each keyword ex) [1, 3]. In most situations, keywords are extracted according to the maximum number of words (list[int, int], optional, default: [1, 2])</li>
                <li>"top_n": number of keywords to be extracted, in order of highest similarity (int, optional, default: 5)</li>
                <li>"diversity": variety of keywords to be extracted. The higher the value, the more diverse the keywords will be 0 <= diversity <= 1 (float, optional, default: 0.5)</li>
                <li>"use_stopwords": whether to exclude words that do not have a significant meaning (bool, optional, True|False, default: True)</li>
                <li>"threshold": minimum value of similarity value of keywords to be extracted (float, optional, default: 0.0)</li>
            </ul>
            </li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, <strong>movie_review_test</strong> table is used.</li>
    </ul>
</div>

## __5. Combine the Two Methods__

In [13]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col='review',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT review, sentiment, score
    FROM (
        SEARCH TEXT 
        USING movie_text_search_model
        OPTIONS (
            search_by='text',
            search_input='The greatest movie of all time',
            emb_col='convert_result',
            result_col='score',
            top_k=10
            )
        AS 
        SELECT * 
        FROM movie_review_test
        )
    )

,review,sentiment,score,keyword
0,This is without a doubt the most poorly though...,negative,0.570813,"{'keyword': ['poorly thought movie', 'movie hi..."
1,This movie was the second movie I saw on the c...,positive,0.568432,"{'keyword': ['movie great monsters', 'second m..."
2,"THE PROTECTOR. You hear the name. You think, ""...",negative,0.567183,"{'keyword': ['hong kong movie', 'better cop fi..."
3,This is a good film. This is very funny. Yet a...,positive,0.560270,"{'keyword': ['good ernest films', 'funny film'..."
4,"First ever viewing: July 21, 2008 Very impress...",positive,0.559803,"{'keyword': ['awards comedies', '2008 impressi..."
5,This is the best Chinese movie I have ever see...,positive,0.555390,"{'keyword': ['best chinese movie', 'makes enjo..."
6,For those out there that like to think of them...,positive,0.545683,"{'keyword': ['modest paced movie', 'love film ..."
7,"First off, I didn't know what to expect when I...",positive,0.539408,"{'keyword': ['cult type movie', 'genre adapts ..."
8,The movie Angels of the Universe is a pure mas...,positive,0.539202,"{'keyword': ['greatest movie iceland', 'film a..."
9,"This movie is full of references. Like ""Mad M...",positive,0.538833,"{'keyword': ['lorre movie masterpiece', 'movie..."


In [14]:
%%thanosql
SELECT review, sentiment, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM ( 
    SEARCH KEYWORD
    USING movie_text_search_model
    OPTIONS (
        text_col='review',
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT review, sentiment, score
        FROM (
            SEARCH TEXT 
            USING movie_text_search_model
            OPTIONS (
                search_by='text',
                search_input='The greatest movie of all time',
                emb_col='convert_result',
                result_col='score',
                top_k=10
                )
            AS 
            SELECT * 
            FROM movie_review_test
            )
        )
    )

,review,sentiment,keywords,score
0,This is without a doubt the most poorly though...,negative,"[poorly thought movie, movie history, history ...","[0.5816, 0.5223, 0.3959, 0.2803, 0.2332]"
1,This movie was the second movie I saw on the c...,positive,"[movie great monsters, second movie saw, scare...","[0.6603, 0.498, 0.3572, 0.3463, 0.1984]"
2,"THE PROTECTOR. You hear the name. You think, ""...",negative,"[hong kong movie, better cop film, stupid jack...","[0.5938, 0.5932, 0.3732, 0.3613, 0.2269]"
3,This is a good film. This is very funny. Yet a...,positive,"[good ernest films, funny film, film good, fil...","[0.6649, 0.6243, 0.5778, 0.5463, 0.2268]"
4,"First ever viewing: July 21, 2008 Very impress...",positive,"[awards comedies, 2008 impressive screenplay, ...","[0.6833, 0.5813, 0.4406, 0.3565, 0.2449]"
5,This is the best Chinese movie I have ever see...,positive,"[best chinese movie, makes enjoyable movie, gh...","[0.6368, 0.5155, 0.4634, 0.4069, 0.3663]"
6,For those out there that like to think of them...,positive,"[modest paced movie, love film good, enjoy ind...","[0.5896, 0.5697, 0.5593, 0.2967, 0.1906]"
7,"First off, I didn't know what to expect when I...",positive,"[cult type movie, genre adapts present, crockz...","[0.6487, 0.4709, 0.3614, 0.2413, 0.1341]"
8,The movie Angels of the Universe is a pure mas...,positive,"[greatest movie iceland, film award year, symb...","[0.672, 0.5562, 0.5294, 0.5096, 0.3237]"
9,"This movie is full of references. Like ""Mad M...",positive,"[lorre movie masterpiece, movie references lik...","[0.66, 0.5854, 0.3026, 0.2958, 0.2324]"


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|audio|video|text|keyword]</strong>" defines the image|audio|video|text data type to search for.</li>
        <li>"<strong>SEARCH KEYWORD</strong>" uses the <strong>movie_text_search_model</strong> as an algorithm to search keywords.</li>
        <li>"<strong>USING</strong>" defines the model used for text vectorization.</li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, <strong>movie_review_test</strong> table is used.</li>
    </ul>
</div>

## __6. In Conclusion__

In this tutorial, we performed text vectorization using movie review data, and similar text search and keyword extraction. As this is a beginner-level tutorial, we focused on the process rather than accuracy. The model's accuracy can be improved by adjusting various options, such as increasing the epoch or dataset size. Create your own model and provide competitive services by combining various unstructured data (image, audio, video, etc.) and structured data with ThanoSQL.

* [How to Upload My Data to the ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/data_upload/)
* [How to Create a Table Using My Data](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [How to Upload My Model to the ThanoSQL Workspace](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/UPLOAD_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">Inquiries About Deploying a Model for Your Own Service</h4>
    <p>If you have any difficulties creating your own model using ThanoSQL or applying it to your services, please feel free to contact us below😊</p>
    <p>For inquiries regarding building an text similarity search models: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>